# Import Libaries

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")

# Model Fitting

In [6]:
# Step 1: Prepare the Dataset
train_csv_file = 'DataTrain.csv'
test_csv_file = 'DataTest.csv'
train_image_folder = os.path.join(os.getcwd(), 'DataTrain') 
test_image_folder = os.path.join(os.getcwd(), 'DataTest')

# Step 2: Load and Preprocess the Dataset
current_directory = os.getcwd()
train_csv_path = os.path.join(current_directory, train_csv_file)
test_csv_path = os.path.join(current_directory, test_csv_file)
train_image_folder_path = os.path.join(current_directory, train_image_folder)
test_image_folder_path = os.path.join(current_directory, test_image_folder)

df_train = pd.read_csv(train_csv_path)
df_test = pd.read_csv(test_csv_path)

train_image_paths = df_train['NameofFile'].apply(lambda x: os.path.join(train_image_folder_path, x)).values
train_labels = df_train['Vehicleregistrationplate'].values

test_image_paths = df_test['Name of File'].apply(lambda x: os.path.join(test_image_folder_path, x)).values

# Label encoding
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)

# Preprocess the images (e.g., resize, normalize, convert to tensors)
image_size = (224, 224)

train_images = []
for image_path in train_image_paths:
    image = load_img(image_path, target_size=image_size)
    image = img_to_array(image)
    image = image / 255.0  # Normalize the image
    train_images.append(image)
train_images = np.array(train_images)

test_images = []
for image_path in test_image_paths:
    image = load_img(image_path, target_size=image_size)
    image = img_to_array(image)
    image = image / 255.0  # Normalize the image
    test_images.append(image)
test_images = np.array(test_images)

# Calculate the number of plate classes
num_classes = len(label_encoder.classes_)

# Step 4: Build the Model Architecture
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Step 5: Train the Model
batch_size = 32
epochs = 10

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs)

# Step 6: Evaluate and Fine-Tune

# Step 7: Test the Model
# Make predictions on test images
predictions = model.predict(test_images)
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

# Print predicted labels
for image_path, predicted_label in zip(test_image_paths, predicted_labels):
    print(f"Image: {image_path} - Predicted Label: {predicted_label}")



Epoch 1/10


2023-06-26 13:11:42.048245: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


25/25 [==============================] - 38s 1s/step - loss: 7.3745 - accuracy: 0.0063
Epoch 2/10
25/25 [==============================] - 45s 2s/step - loss: 6.8648 - accuracy: 0.0213
Epoch 3/10
25/25 [==============================] - 41s 2s/step - loss: 6.4674 - accuracy: 0.0188
Epoch 4/10
25/25 [==============================] - 40s 2s/step - loss: 6.1520 - accuracy: 0.0175
Epoch 5/10
25/25 [==============================] - 48s 2s/step - loss: 5.8629 - accuracy: 0.0562
Epoch 6/10
25/25 [==============================] - 33s 1s/step - loss: 5.4167 - accuracy: 0.0750
Epoch 7/10
25/25 [==============================] - 32s 1s/step - loss: 4.8584 - accuracy: 0.1138
Epoch 8/10
25/25 [==============================] - 32s 1s/step - loss: 4.2629 - accuracy: 0.1988
Epoch 9/10
25/25 [==============================] - 32s 1s/step - loss: 3.7633 - accuracy: 0.2450
Epoch 10/10
4/4 [==============================] - 3s 489ms/step
Image: /Users/ingwerludwig/DATA/College/6TH TERM/DATA COMPETITIO